In [23]:
import pandas as pd
import json
from pyspark.sql import SparkSession, functions as F

In [3]:

spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/11 16:14:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/11 16:14:38 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
df = spark.read.json("../work-dir/logs/spark-1.json")

24/05/11 16:15:56 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [7]:
# df.printSchema()
df.select("Event").groupBy("Event").count().show(20,False)

+--------------------------------------------------------------------------+-----+
|Event                                                                     |count|
+--------------------------------------------------------------------------+-----+
|org.apache.spark.sql.execution.ui.SparkListenerSQLExecutionEnd            |3    |
|SparkListenerTaskStart                                                    |2115 |
|SparkListenerBlockManagerAdded                                            |4    |
|SparkListenerJobStart                                                     |19   |
|SparkListenerResourceProfileAdded                                         |1    |
|SparkListenerLogStart                                                     |1    |
|SparkListenerExecutorAdded                                                |3    |
|org.apache.spark.sql.execution.ui.SparkListenerSQLExecutionStart          |3    |
|SparkListenerEnvironmentUpdate                                            |1    |
|Spa

In [16]:
df.filter("Event='SparkListenerTaskEnd'").show(10)

+------+--------+----------------+-----------------+---------------+--------------------+-----------+-------------+--------------------------+-----------------+---------------+------+----------+--------------+----------------------+---------------------+------------------+----------+-------------------+----------------+-------------+----------------+--------+---------+----------+-----------+---------------+-----------------+---------------+---------------------+--------------------+--------------------+----------------------+----------+---------+----+------------+-----------+-------+------------+-----------+-----------------------+---------------+-------------+--------------+----+
|App ID|App Name|Block Manager ID|Classpath Entries|Completion Time|               Event|Executor ID|Executor Info|Executor Resource Requests|Hadoop Properties|JVM Information|Job ID|Job Result|Maximum Memory|Maximum Offheap Memory|Maximum Onheap Memory|Metrics Properties|Properties|Resource Profile Id|Spark 

In [17]:
df_tasks = df.filter("Event='SparkListenerTaskEnd'").select("`Stage ID`", "`Task End Reason`.*", "`Task Info`.*", "`Task Metrics`.*")

- durations are in ms
- data sizes are in KiB

In [21]:
df_tasks.filter("`Executor Run Time`>100").show(1000)

+--------+-------+--------------------+-------+-----------+------+-------------+-------------------+-----------+-----+------+-------------+-------------+------------+-----------+-------+------------------+-----------------+-----------------------------+-------------------------+-----------------+-----------------+-----------+--------------------+--------------+---------------------+-------------------------+-----------+--------------------+---------------------+--------------+
|Stage ID| Reason|        Accumulables|Attempt|Executor ID|Failed|  Finish Time|Getting Result Time|       Host|Index|Killed|  Launch Time|     Locality|Partition ID|Speculative|Task ID|Disk Bytes Spilled|Executor CPU Time|Executor Deserialize CPU Time|Executor Deserialize Time|Executor Run Time|    Input Metrics|JVM GC Time|Memory Bytes Spilled|Output Metrics|Peak Execution Memory|Result Serialization Time|Result Size|Shuffle Read Metrics|Shuffle Write Metrics|Updated Blocks|
+--------+-------+------------------

In [28]:
spark.sql("CREATE DATABASE IF NOT EXISTS test")
spark.sql("USE DATABASE test")
df1 = spark.range(1000)
df1_random = df1.select((F.rand() * 1000).alias("number"))
df1_random.write.mode("overwrite").saveAsTable("random_numbers")

In [31]:
spark.sql("SELECT * FROM random_numbers").show(10)
spark.sql("SHOW TABLES").show()

+------------------+
|            number|
+------------------+
|360.95863169709895|
| 384.9855891399047|
| 540.4944876784583|
| 538.4187785778086|
|305.82313570682663|
|158.00541027792204|
| 925.1413456793454|
| 706.7315122776914|
|  565.068691128227|
| 896.9770913617226|
+------------------+
only showing top 10 rows

+---------+--------------+-----------+
|namespace|     tableName|isTemporary|
+---------+--------------+-----------+
|     test|random_numbers|      false|
+---------+--------------+-----------+

